In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE DATABASE snowflakegold;
USE SCHEMA gold_layer;

In [ ]:
SHOW TABLES IN SCHEMA gold_layer;

In [ ]:
query = "select * from sales_order_details"

In [ ]:
df = session.sql(query).to_pandas()

In [ ]:
df

In [ ]:
X = df[['UNIT_PRICE', 'PRODUCT_ID']]
y = df['ORDER_QTY']  # This represents demand

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),  # Input layer
    tf.keras.layers.Dense(64, activation='relu'),               # Hidden layer 1
    tf.keras.layers.Dense(32, activation='relu'),               # Hidden layer 2
    tf.keras.layers.Dense(1)                                    # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {test_loss}')

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test_scaled)
# Round off the predictions
rounded_predictions = np.round(predictions).astype(int)  # Convert to integers if necessary
# Convert predictions to a DataFrame for easy comparison
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': rounded_predictions.flatten()
})

# Display the first few rows of the predictions
print(predictions_df.head())
